In [ ]:
from keras.applications.mobilenet import MobileNet, preprocess_input, relu6
from keras.models import Model, load_model
from keras.layers import Input,  Dense, GlobalAveragePooling2D, BatchNormalization
from keras.optimizers import Adam
from keras import backend as K
from keras.callbacks import TensorBoard, ModelCheckpoint
import matplotlib.pyplot as plt

In [ ]:
import sys
sys.path.append("../../../util/keras_util")
sys.path.append("../../../util")
from coco_obj_generator import CocoObjGenerator

In [ ]:
img_dir =  '/datasets/sbr/posture_recognition/images'
train_label = '/datasets/sbr/posture_recognition/annotations/train.json'
val_label = '/datasets/sbr/posture_recognition/annotations/val.json'
label_text = ['others', 'lie', 'sit', 'stand']
# other, lie, sit, stand
num_label = 4
batch_size = 256
num_of_epochs = 350

In [ ]:
im_width, im_height = 224, 224
base_model = MobileNet(alpha=0.25, include_top=False)

## Define network

In [ ]:
# Add new layer after MobileNet
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(num_label, activation='softmax', name='fc_predict')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
opt = Adam(lr=0.0001, decay=1e-6)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

## Data Augmentation

In [ ]:
train_data_aug = CocoObjGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    bbox_width_shift_range=0.1,
    bbox_height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
)


train_generator = train_data_aug.flow_from_directory(
    img_dir, train_label,
    color_mode='random',
    target_size=(im_width, im_height),
    batch_size=batch_size,
)

val_data_aug = CocoObjGenerator(
    preprocessing_function=preprocess_input,
    bbox_width_shift_range=0.1,
    bbox_height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
)


val_generator = val_data_aug.flow_from_directory(
    img_dir, val_label,
    target_size=(im_width, im_height),
    batch_size=batch_size,
)

## Save model, Plot learning curve

In [ ]:
# Plot learning curve on tensor board
tensor_board = TensorBoard(log_dir='/log/tensorboard', batch_size=batch_size)

# Save best model
save_model = ModelCheckpoint("weights.{epoch:02d}-{acc:.2f}-{loss:.4f}.hdf5",
                                 monitor="val_loss",
                                 verbose=1,
                                 save_best_only=True,
                                 mode="auto")

## Gradient descent

In [ ]:
# Fit the model on the batches generated by datagen.flow().
# model.load_weights('weights.49-0.72-0.7661.hdf5')
model.fit_generator(train_generator, 
                    epochs = num_of_epochs,
                    callbacks = [tensor_board, save_model],
                    validation_data = val_generator,
                    initial_epoch=0)

# Save the best model
# model = load_model('weights.299-1.00-0.0129.hdf5', custom_objects={'relu6': relu6})
# model.save('posture_model.hdf5')

## Model convertion - Keras to Tensorflow

In [ ]:
model.save('posture_recog.hdf5')
from keras_to_tensorflow import save_frozen_tf_model
save_frozen_tf_model(model, 'posture_recog.pb')

## Experiment - precision, recall, F-score

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

expirement_generator = val_data_aug.flow_from_directory(
    img_dir, val_label,
    color_mode='rgb',
    target_size=(im_width, im_height),
    batch_size=batch_size,
)

x_test, y_test = expirement_generator.next()
y_pred = model.predict(x_test)
y_test = np.argmax(y_test, axis=1) # Convert one-hot to index
y_pred = np.argmax(y_pred, axis=1) # Convert one-hot to index

print(classification_report(y_test, y_pred))

## Inference network

In [ ]:
demo_generator = val_data_aug.flow_from_directory(
    img_dir, val_label,
    color_mode='random',
    target_size=(im_width, im_height),
    batch_size=4,
)

In [ ]:
imgs, labels = demo_generator.next()
cols = 4
rows = len(imgs) / cols
f = plt.figure(figsize=(15, 15))

for i in range(len(imgs)):
    plt.subplot(rows, cols , i + 1)
    plt.axis('Off')
    img = (imgs[i] + 1 ) / 2
    plt.imshow(img)
    print(label_text[np.argmax(labels[i])])

plt.show()

In [ ]:
img = (imgs[0] + 1 ) / 2
plt.imshow(img)
plt.show()
img = imgs[0].reshape(1, 224, 224, 3)
label = model.predict(img)
print (label)
print(label_text[np.argmax(label)])